In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aicsimageio import AICSImage
import time

### package for 3d visualization
#from itkwidgets import view                              
#from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
#import skimage

### local new python segmentation functions
import os
import sys
pythonPackagePath = os.path.abspath('../src/')
sys.path.append(pythonPackagePath)
from peak_local_max_3d import peak_local_max_3d 
from gaussian_fitting import fit_multiple_gaussians
from extract_data import extract_data_from_filename
from gaussian_visualization import visualize_3D_gaussians
from gaussian_fitting import check_fitting_error
import matplotlib.pyplot as plt
from skimage import io
import tifffile

In [2]:

# input_file_path = '/Users/makamats/Documents/Local-microscopy-data/TwoSlide_Left_JFXRapidWithWash-05_processed-cropped.ome.tiff'
input_file_path = '/Users/makamats/Documents/Local-microscopy-data/TwoSlide_Left_JFXRapidWithWash-05_processed-cropped-double-crop.ome.tif'
save_dir = '/Users/makamats/Documents/Local-microscopy-data/Tracked'

# Load the file
img = AICSImage(input_file_path)

In [3]:
# Check which reader is being used
print(type(img.reader))

<class 'aicsimageio.readers.tiff_reader.TiffReader'>


In [4]:
img.dask_data

dask.array<transpose, shape=(130, 3, 75, 150, 275), dtype=uint16, chunksize=(1, 1, 75, 150, 275), chunktype=numpy.ndarray>

In [23]:
import time

# Load the CZI file
#img = AICSImage(input_filename_czi)

file_label = input_file_path.split('/')[-1].split('.')[0]

#get the number of frames for our original data for automated analysis for all frames 
frames = img.dims.T
start_frame = 0
end_frame = frames

# channels = img.dims.C

threshold = [150]
### an otsu or other type of threshold might be better here

start_time = time.time()

# Single channel detection, to start

# pick which channel to detect (starts at 0)
channel = 2

# for channel in range(channels):
#     print('processing channel ' + str(channel))
### This is where we can replace deep learning detection, or parallel detection on multiple CPUs for example

# input_file_path

detections_df = pd.DataFrame()

for frame in range(start_frame,end_frame):
    
    print('frame number is', frame)
    lazy_single_frame_input = img.get_image_dask_data("ZYX", T=frame, C=channel)
    
    single_frame_input = lazy_single_frame_input.compute()
    print(single_frame_input.shape)
    single_frame_input = np.transpose(single_frame_input,axes =(0,2,1))
    print(single_frame_input.shape)

    #define threshold: a value(intensity) for the pixel below which all values would be considered noise and dropped 
    #define min_distance: min_distance/2 is the radius within which we will keep the peak with max value/intensity or 
    #if two peaks have the same value they will be kept 
    
    ### we should be able to determine the threshold intelligently, i.e. with an otsu filter or some other method, for each
    maximas = peak_local_max_3d(single_frame_input,min_distance=10,threshold=threshold)
    print('local_maximas detected are', maximas.shape[0])


    #give the expected std dev/radius of our particles for x,y,z 
    sigmaExpected_x__pixels = 2
    sigmaExpected_y__pixels = 2
    sigmaExpected_z__pixels = 4

    sigmas_guesses = []
    for i in range(len(maximas)):
        sigmas_guesses.append([sigmaExpected_z__pixels,sigmaExpected_x__pixels,sigmaExpected_y__pixels])
        
    #last parameter in the fit_multiple_gaussians is similar to min_distance above, we should give half of the 
    #value here of min_distance   
    gaussians, gaussians_popt = fit_multiple_gaussians(single_frame_input,maximas,sigmas_guesses,5)
        
    accumulator = []
    for gaussian in gaussians:

        if(gaussian!=-1):
            amplitude = gaussian[0]

            #print(gaussian)
            mu_x     = int(gaussian[1][1]) ##this is going to be mu_z, previous code [1][0]
            mu_y     = int(gaussian[1][2]) ##need to finalise what this is (x or y) [1][1]
            mu_z     = int(gaussian[1][0]) ##need to finalise what this is (x or y) [1][2]
            ##sigmas will also change due to the above 
            sigma_x  = int(gaussian[2][1]) 
            sigma_y  = int(gaussian[2][2])
            sigma_z  = int(gaussian[2][0])
            accumulator.append(np.array([amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z]))
            
    accumulator = np.array(accumulator)
    print(accumulator.shape)
    df = pd.DataFrame()
    df['amplitude'] = accumulator[:,0]
    df['mu_x'] = accumulator[:,1]
    df['mu_y'] = accumulator[:,2]
    df['mu_z'] = accumulator[:,3]
    df['sigma_x'] = accumulator[:,4]
    df['sigma_y'] = accumulator[:,5]
    df['sigma_z'] = accumulator[:,6]
    df['channel'] = channel
    df['frame'] = frame
    # df.head()
    
    error_list, index_list = check_fitting_error(single_frame_input,maximas,gaussians,sigmas_guesses)
    
    # Construct the filename
    '''
    filename_csv = f'df_c2_t{frame}.csv'
    file_path_csv = os.path.join(csv_save_dir, filename_csv)
    df.to_csv(file_path_csv)
    '''
    # concatenate the dataframes

    detections_df = pd.concat([detections_df,df])
    
# Save the DataFrame to a pickle file with the specified path
file_path =   os.path.join(save_dir,file_label+'_detections_c'+str(channel)+'.pkl')

detections_df.to_pickle(file_path)
end_time = time.time()

print('time taken (seconds)', end_time - start_time)

frame number is 0
(75, 150, 275)
(75, 275, 150)
local_maximas detected are 456
10%(46 of 456)
20%(92 of 456)
30%(137 of 456)
40%(183 of 456)
50%(228 of 456)
60%(274 of 456)
70%(320 of 456)
80%(365 of 456)
90%(411 of 456)
100%(456 of 456)
(456, 7)
the number of times the gaussian fitting worked was456and the number of times the gaussian did not fit was 0
frame number is 1
(75, 150, 275)
(75, 275, 150)
local_maximas detected are 446
10%(45 of 446)
20%(90 of 446)
30%(134 of 446)
40%(179 of 446)
50%(223 of 446)
60%(268 of 446)
70%(313 of 446)
80%(357 of 446)
90%(402 of 446)
100%(446 of 446)
(446, 7)
the number of times the gaussian fitting worked was446and the number of times the gaussian did not fit was 0
frame number is 2
(75, 150, 275)
(75, 275, 150)
local_maximas detected are 464
10%(47 of 464)
20%(93 of 464)
30%(140 of 464)
40%(186 of 464)
50%(232 of 464)
60%(279 of 464)
70%(325 of 464)
80%(372 of 464)
90%(418 of 464)
100%(464 of 464)
(464, 7)
the number of times the gaussian fitting 